### Testing particle tracking to fix virial masses of satellites.

In [1]:
from base import *

In [2]:
 def calc_snGas(sim, haloid, save=True, verbose=True):
    '''
    -> Identifies all gas particles that were subject to supernova heating in the
        simulations.
    '''
    #--------------------------------#
    
    import tqdm
    data = read_tracked_particles(sim, haloid, verbose=verbose)
    
    if verbose: print(f'Now compiling SN-heated gas for {sim}-{haloid}...')
    
    sngas = pd.DataFrame() # all gas in sims that experienced SN-heating.
    
    pids = np.unique(data.pid)
    for pid in tqdm.tqdm(pids):
        dat = data[data.pid==pid]

        time = np.array(dat.time, dtype=float)
        coolontime = np.array(dat.coolontime, dtype=float)
        
        for i,t2 in enumerate(time[1:]):
                if (coolontime[i] > time[i]):
                    hot = dat[time==t2].copy()
                    sngas = pd.concat([sngas, hot])
                i += 1
    
    if save:
        key = f'{sim}_{str(int(haloid))}'
        filepath = f'{rootPath}Stellar_Feedback_Code/SNeData/sngas_particles.hdf5'
        print(f'Saving {key} SN-heated particles to {filepath}')
        sngas.to_hdf(filepath, key=key)
        
    print(f'> Returning (SNgas) dataset <')
    return sngas

In [16]:
def read_tracked_particles(sim, haloid, verbose=False):
    '''
    -> Reads in gas particles tracked across a number of simulation satellites and calculates/appends desired particle 
        properties for analysis.
    '''
    #--------------------------------#
    
    if verbose: print(f'Loading tracked particles for {sim}-{haloid}...')
    
    key = f'{sim}_{str(int(haloid))}'

    # importing tracked particles.
    # 'v2' revision contains all tracked particles for the 19 satellite halos selected for this study (see README):
    path1 = f'{rootPath}Stellar_Feedback_Code/SNeData/tracked_particles.hdf5'
    data = pd.read_hdf(path1, key=key)
    
    time = np.unique(data.time)
    dt = time[1:]-time[:-1]
    dt = np.append(dt[0], dt)
    dt = dt[np.unique(data.time, return_inverse=True)[1]]
    data['dt'] = dt
    
    
    if verbose: print('Successfully loaded')
    
    r_gal = np.array([])
    for t in np.unique(data.time):
        d = data[data.time==t]
        r_gas = np.mean(d.sat_r_gas)
        r_half = np.mean(d.sat_r_half)
        rg = np.max([r_gas,r_half])

        if np.isnan(rg):
            rg = r_gal_prev

        if verbose: print(f't = {t:1f} Gyr, satellite R_gal = {rg:.2f} kpc')
        r_gal = np.append(r_gal,[rg]*len(d))

        r_gal_prev = rg

    data['r_gal'] = r_gal
    
    r_gal_prev = 0
    r_gal = np.array([])
    for t in np.unique(data.time):
        d = data[data.time==t]
        r_gas = np.mean(d.host_r_gas)
        r_half = np.mean(d.host_r_half)
        rg = np.max([r_gas,r_half])

        if np.isnan(rg):
            rg = r_gal_prev

        if verbose: print(f't = {t:1f} Gyr, host R_gal = {rg:.2f} kpc')
        r_gal = np.append(r_gal,[rg]*len(d))

        r_gal_prev = rg

    data['host_r_gal'] = r_gal
    
    thermo_disk = (np.array(data.temp) < 1.2e4) & (np.array(data.rho) > 0.1)
    
    in_sat = np.array(data.in_sat)
    other_sat = np.array(data.in_other_sat)
    in_host = np.array(data.in_host) & ~in_sat & ~other_sat
    
    sat_disk = in_sat & thermo_disk
    sat_halo = in_sat & ~thermo_disk
    
    host_disk = in_host & thermo_disk
    host_halo = in_host & ~thermo_disk
    
    IGM = np.array(data.in_IGM)
    
    # basic location classifications.
    data['sat_disk'] = sat_disk
    data['sat_halo'] = sat_halo
    data['host_disk'] = host_disk
    data['host_halo'] = host_halo
    data['other_sat'] = other_sat
    data['IGM'] = IGM

    # appending the virial mass of a particle's respective satellite.
    timesteps = read_timesteps(sim)
    ts = timesteps[timesteps.z0haloid==haloid]
    ts = ts.rename({'mass':'sat_Mvir'}, axis=1)
    ts = ts[['time','sat_Mvir']]

    ts['sat_Mvir'] = ts['sat_Mvir'].astype('float')

    data = pd.merge_asof(data, ts.sort_values('time'), left_on='time', right_on='time', direction='nearest', tolerance=1)

    return data

In [20]:
read_tracked_particles('h148', '13')

,time,pid,rho,temp,mass,coolontime,r,r_per_Rvir,x,y,...,dt,r_gal,host_r_gal,sat_disk,sat_halo,host_disk,host_halo,other_sat,IGM,sat_Mvir
0,6.474131,2202002,0.000050,619653.812500,27934.820603,1.502397,315.430011,7.446391,301.379304,38.695749,...,0.43098,0.838365,18.443250,False,False,False,False,False,True,NaN
1,6.474131,2205223,0.000034,246611.796875,26567.141174,0.000000,345.770111,8.162633,283.107089,38.728024,...,0.43098,0.838365,18.443250,False,False,False,False,False,True,NaN
2,6.474131,2857672,0.000074,708055.187500,26511.576364,0.000000,373.134629,8.808630,360.627617,-35.282347,...,0.43098,0.838365,18.443250,False,False,False,True,False,False,NaN
3,6.474131,4057198,0.000023,405099.468750,26510.536704,0.000000,593.153339,14.002636,563.303480,-183.727503,...,0.43098,0.838365,18.443250,False,False,False,False,False,True,NaN
4,6.474131,4073949,0.000024,405991.656250,26510.536704,0.000000,588.527236,13.893427,556.331718,-188.298945,...,0.43098,0.838365,18.443250,False,False,False,False,False,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299100,13.800797,19009255,0.478361,733.537537,30288.071656,13.796250,284.462060,4.318511,49.714790,250.171752,...,0.43098,0.858178,14.708206,False,False,True,False,False,False,NaN
299101,13.800797,19078060,0.000206,840771.562500,26645.618222,8.779381,390.337777,5.925844,67.699234,345.184330,...,0.43098,0.858178,14.708206,False,False,False,True,False,False,NaN
299102,13.800797,19100998,0.000036,204305.203125,26859.950488,8.804634,85.520235,1.298310,-53.786871,36.639670,...,0.43098,0.858178,14.708206,False,False,False,True,False,False,NaN
299103,13.800797,19103645,0.000005,238948.421875,28078.179678,11.644720,154.169322,2.340494,129.548087,-47.754756,...,0.43098,0.858178,14.708206,False,False,False,True,False,False,NaN


In [ ]:
hal, data = read_tracked_particles('h148', '28')
np.unique(hal)